## Doing Imports and Preprocessing

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

from sklearn.neural_network import MLPRegressor

In [ ]:
train_data = pd.read_csv("../input/tabular-playground-series-aug-2021/train.csv")
valid_data = pd.read_csv("../input/tabular-playground-series-aug-2021/test.csv")

In [ ]:
train_data.drop('id', axis=1, inplace=True)

X = train_data.drop('loss', axis=1)
y = train_data['loss']

In [ ]:
ss = StandardScaler()
X = ss.fit_transform(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42)

In [ ]:
valid_ids = valid_data['id']
valid_data = valid_data.drop('id',axis=1)

In [ ]:
valid_data = ss.fit_transform(valid_data)

## Training the model and Tuning it with GridSearchCV

In [ ]:
import multiprocessing
n_jobs = multiprocessing.cpu_count()-1
n_jobs

In [ ]:
mlp = MLPRegressor()
param_grid = {'hidden_layer_sizes': [i for i in range(2,20)],
              'activation': ['relu'],
              'solver': ['adam'],
              'learning_rate': ['constant'],
              'learning_rate_init': [0.01],
              'power_t': [0.5],
              'alpha': [0.0001],
              'max_iter': [1000],
              'early_stopping': [True],
              'warm_start': [False]}
mlp_GS = GridSearchCV(mlp, param_grid=param_grid, 
                   cv=10, verbose=True, pre_dispatch='2*n_jobs', n_jobs=n_jobs)

In [ ]:
mlp_GS.fit(X_train, y_train)

## Making Predictions and Exporting Results

In [ ]:
p_hat = mlp_GS.predict(X_train)

In [ ]:
mean_squared_error(y_train, p_hat, squared=False)

In [ ]:
sample_sub = pd.read_csv("../input/tabular-playground-series-aug-2021/sample_submission.csv")

In [ ]:
valid_loss = mlp_GS.predict(valid_data)

In [ ]:
sample_sub.loss = valid_loss

In [ ]:
sample_sub.to_csv('mlp_loss.csv', index=False)

## Conclusion

No. The Neural Network based `MLPRegressor` scorred signifcantly lower than the `XGBoost`, as I have done in [this notebook](https://www.kaggle.com/hrbukhari/optuna-tuned-boosting-models-cat-xgb).